In [1]:
import sys
sys.path.insert(0,'/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg')
sys.path.insert(0,'/exp/rhuang/meta/k2/k2/python')
sys.path.insert(0,'/exp/rhuang/meta/k2/temp.linux-x86_64-cpython-310/lib')
sys.path.insert(0,'/exp/rhuang/meta/icefall')

In [2]:
%cd /exp/rhuang/meta/icefall/egs/librispeech/ASR
%pwd  #look at the current work dir

/exp/rhuang/meta/icefall/egs/librispeech/ASR


'/exp/rhuang/meta/icefall/egs/librispeech/ASR'

In [3]:
import argparse
import logging
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import k2
import sentencepiece as spm
import torch
import torch.nn as nn
from asr_datamodule import LibriSpeechAsrDataModule
from conformer import Conformer

from icefall.bpe_graph_compiler import BpeCtcTrainingGraphCompiler
from icefall.checkpoint import (
    average_checkpoints,
    average_checkpoints_with_averaged_model,
    find_checkpoints,
    load_checkpoint,
)
from icefall.decode import (
    get_lattice,
    nbest_decoding,
    nbest_oracle,
    one_best_decoding,
    rescore_with_attention_decoder,
    rescore_with_n_best_list,
    rescore_with_rnn_lm,
    rescore_with_whole_lattice,
)
from icefall.env import get_env_info
from icefall.lexicon import Lexicon
from icefall.rnn_lm.model import RnnLmModel
from icefall.utils import (
    AttributeDict,
    get_texts,
    load_averaged_model,
    setup_logger,
    store_transcripts,
    str2bool,
    write_error_stats,
)
# from train import get_params as get_train_params
from decode import get_parser, get_params
from icefall.decode import get_lattice, one_best_decoding
from icefall.utils import get_alignments, get_texts, get_texts_with_timestamp

import psutil
try:
    from tqdm_loggable.auto import tqdm
except:
    from tqdm import tqdm
import torch.multiprocessing as mp

/home/hltcoe/rhuang/mambaforge/envs/aligner5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
parser = get_parser()
LibriSpeechAsrDataModule.add_arguments(parser)
args = parser.parse_args(args=[])

args.exp_dir = "/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg/exp/exp_seed"
args.epoch = 14
args.avg = 1
args.use_averaged_model = False

args.manifest_dir = "/exp/rhuang/meta/icefall/egs/librispeech/ASR/data/fbank/"
args.max_duration = 30
args.world_size = 1
args.method = 'ctc-decoding'

args.exp_dir = Path(args.exp_dir)
args.lang_dir = Path(args.lang_dir)
args.lm_dir = Path(args.lm_dir)
args.manifest_dir = Path(args.manifest_dir)

params = get_params()
params.update(vars(args))

device = torch.device("cpu")

sp = spm.SentencePieceProcessor()
sp.load(params.bpe_model)

# <blk> is defined in local/train_bpe_model.py
params.blank_id = sp.piece_to_id("<blk>")
params.vocab_size = sp.get_piece_size()

In [5]:
lexicon = Lexicon(params.lang_dir)
max_token_id = max(lexicon.tokens)
num_classes = max_token_id + 1  # +1 for the blank

if params.method == "ctc-decoding" or params.method == "ctc-greedy-search":
    HLG = None
    H = k2.ctc_topo(
        max_token=max_token_id,
        modified=False,
        device=device,
    )
    bpe_model = spm.SentencePieceProcessor()
    bpe_model.load(str(params.lang_dir / "bpe.model"))
else:
    H = None
    bpe_model = None
    HLG = k2.Fsa.from_dict(
        torch.load(f"{params.lang_dir}/HLG.pt", map_location=device)
    )
    assert HLG.requires_grad is False

    if not hasattr(HLG, "lm_scores"):
        HLG.lm_scores = HLG.scores.clone()

print(f"H: {H is not None}")
print(f"HLG: {HLG is not None}")

H: True
HLG: False


In [6]:
print(params)

{'subsampling_factor': 4, 'feature_dim': 80, 'nhead': 8, 'dim_feedforward': 2048, 'encoder_dim': 512, 'num_encoder_layers': 12, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '2b2ac14b326d61d79d04e53fbd69b1ff6d630411', 'k2-git-date': 'Wed Aug 23 17:58:26 2023', 'lhotse-version': '1.20.0.dev+git.c863fa72.clean', 'torch-version': '2.0.1', 'torch-cuda-available': False, 'torch-cuda-version': '11.7', 'python-version': '3.1', 'icefall-git-branch': 'context', 'icefall-git-sha1': '986d7cc-dirty', 'icefall-git-date': 'Sat Feb 10 18:57:23 2024', 'icefall-path': '/exp/rhuang/meta/icefall', 'k2-path': '/exp/rhuang/meta/k2/k2/python/k2/__init__.py', 'lhotse-path': '/exp/rhuang/meta/lhotse/lhotse/__init__.py', 'hostname': 'test2', 'IP address': '10.1.250.202'}, 'blank_id': 0, 'vocab_size': 500, 'epoch': 14, 'iter': 0, 'avg': 1, 'me

In [7]:
# params = AttributeDict(
#     {'best_train_loss': float('inf'), 'best_valid_loss': float('inf'), 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 50, 'reset_interval': 200, 'valid_interval': 3000, 'feature_dim': 80, 'subsampling_factor': 4, 'warm_step': 2000, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '415fe1f446fffe1d9e7219b5033966294c0b430c', 'k2-git-date': 'Wed Dec 20 21:38:57 2023', 'lhotse-version': '1.20.0.dev+git.c6788492.clean', 'torch-version': '2.1.2', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'noseg', 'icefall-git-sha1': 'b0ecb7f-dirty', 'icefall-git-date': 'Thu Feb 1 03:39:11 2024', 'icefall-path': '/scratch4/skhudan1/rhuang25/icefall', 'k2-path': '/scratch4/skhudan1/rhuang25/k2/k2/python/k2/__init__.py', 'lhotse-path': '/scratch4/skhudan1/rhuang25/lhotse/lhotse/__init__.py', 'hostname': 'gpu02', 'IP address': '172.20.30.2'}, 'frame_shift_ms': 10, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 1, 'iter': 0, 'avg': 1, 'use_averaged_model': False, 'exp_dir': Path('zipformer_no_seg/exp-test'), 'bpe_model': 'data/lang_bpe_500/bpe.model', 'lang_dir': Path('data/lang_bpe_500'), 'context_size': 2, 'decoding_method': '1best', 'num_paths': 100, 'nbest_scale': 1.0, 'hlg_scale': 0.6, 'lm_dir': Path('data/lm'), 'num_encoder_layers': '2,2,3,4,3,2', 'downsampling_factor': '1,2,4,8,4,2', 'feedforward_dim': '512,768,1024,1536,1024,768', 'num_heads': '4,4,4,8,4,4', 'encoder_dim': '192,256,384,512,384,256', 'query_head_dim': '32', 'value_head_dim': '12', 'pos_head_dim': '4', 'pos_dim': 48, 'encoder_unmasked_dim': '192,192,256,256,256,192', 'cnn_module_kernel': '31,31,15,15,15,31', 'decoder_dim': 512, 'joiner_dim': 512, 'causal': False, 'chunk_size': '16,32,64,-1', 'left_context_frames': '64,128,256,-1', 'use_transducer': False, 'use_ctc': True, 'full_libri': True, 'mini_libri': False, 'manifest_dir': Path('data/fbank'), 'max_duration': 500, 'bucketing_sampler': True, 'num_buckets': 30, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': True, 'drop_last': True, 'return_cuts': True, 'num_workers': 2, 'enable_spec_aug': True, 'spec_aug_time_warp_factor': 80, 'enable_musan': True, 'input_strategy': 'PrecomputedFeatures', 'res_dir': Path('zipformer_no_seg/exp-test/1best'), 'suffix': 'epoch-1-avg-1', 'vocab_size': 500, 'blank_id': 0}
# )

In [8]:
logging.info("About to create model")
model = Conformer(
    num_features=params.feature_dim,
    nhead=params.nhead,
    d_model=params.encoder_dim,
    num_classes=params.vocab_size,
    subsampling_factor=params.subsampling_factor,
    num_encoder_layers=params.num_encoder_layers,
    num_decoder_layers=params.num_decoder_layers,
)

load_checkpoint(f"{params.exp_dir}/epoch-{params.epoch}.pt", model)
model.to(device)

num_param = sum([p.numel() for p in model.parameters()])
logging.info(f"Number of model parameters: {num_param}")

In [9]:
# _ = model.eval()
_ = model.train()

In [10]:
args.return_cuts = True
librispeech = LibriSpeechAsrDataModule(args)

train_cuts = librispeech.train_clean_100_cuts()
# if params.full_libri:
#     train_cuts += librispeech.train_clean_360_cuts()
#     train_cuts += librispeech.train_other_500_cuts()
# train_cuts = librispeech.train_other_500_cuts()
train_cuts = train_cuts.to_eager()

# valid_cuts = librispeech.dev_clean_cuts()
# valid_cuts += librispeech.dev_other_cuts()

# test_cuts = librispeech.test_clean_cuts()
# test_cuts += librispeech.test_other_cuts()
# test_cuts = test_cuts.to_eager()

In [70]:
# c = train_cuts.sample(n_cuts=1)
c = train_cuts['3242-8112-0039-24096_sp0.9']
print(c)
print(c.supervisions[0].text)

MonoCut(id='3242-8112-0039-24096_sp0.9', start=0.0, duration=15.7166875, channel=0, supervisions=[SupervisionSegment(id='3242-8112-0039_sp0.9', recording_id='3242-8112-0039_sp0.9', start=0.0, duration=15.7166875, channel=0, text='SO TRIGGER SAID THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM SAID THE COMMISSIONER AROUND FORTY', language='English', speaker='3242', gender=None, custom=None, alignment=None)], features=Features(type='kaldi-fbank', num_frames=1572, num_features=80, frame_shift=0.01, sampling_rate=16000, start=0.0, duration=15.7166875, storage_type='lilcom_chunky', storage_path='data/fbank/librispeech_feats_train-clean-100/feats-0.lca', storage_key='431182223,45050,45021,44919,6674', recording_id='None', channels=0), recording=Recording(id='3242-8112-0039_sp0.9', sources=[AudioSource(type='file', channels=[0], source='/exp/rhuang/librispeech/download2/LibriSpeech/

In [71]:
c.play_audio()

In [14]:
from factor_transducer import *

In [16]:
libri_long_text_str, libri_long_text_fst = get_long_text(train_cuts, sp=sp, make_fst=True, nj=6)
# libri_long_text_str, libri_long_text_fst = get_long_text([c], sp=sp, make_fst=True)
# libri_long_text_str, libri_long_text_fst = get_long_text([train_cuts['4340-15220-0045-15863'], train_cuts['4340-15220-0045-15863_sp0.9']], sp=sp, make_fst=True)
libri_long_text_str = {k: v.split() for k, v in libri_long_text_str.items()}

first_item_key = list(libri_long_text_fst.items())[0][0]
print(libri_long_text_str[first_item_key])
fst = libri_long_text_fst[first_item_key]
print(fst.shape[0], fst.num_arcs)

100%|██████████| 585/585 [00:23<00:00, 24.99it/s]


['ABOVE', 'THIS', 'COUPLE', 'THERE', 'WAS', 'ANNE', 'QUEEN', 'OF', 'ENGLAND', 'AN', 'ORDINARY', 'WOMAN', 'WAS', 'QUEEN', 'ANNE', 'SHE', 'WAS', 'GAY', 'KINDLY', 'AUGUST', 'TO', 'A', 'CERTAIN', 'EXTENT', 'NO', 'QUALITY', 'OF', 'HERS', 'ATTAINED', 'TO', 'VIRTUE', 'NONE', 'TO', 'VICE', 'HER', 'STOUTNESS', 'WAS', 'BLOATED', 'HER', 'FUN', 'HEAVY', 'HER', 'GOOD', 'NATURE', 'STUPID', 'SHE', 'WAS', 'STUBBORN', 'AND', 'WEAK', 'AS', 'A', 'WIFE', 'SHE', 'WAS', 'FAITHLESS', 'AND', 'FAITHFUL', 'HAVING', 'FAVOURITES', 'TO', 'WHOM', 'SHE', 'GAVE', 'UP', 'HER', 'HEART', 'AND', 'A', 'HUSBAND', 'FOR', 'WHOM', 'SHE', 'KEPT', 'HER', 'BED', 'AS', 'A', 'CHRISTIAN', 'SHE', 'WAS', 'A', 'HERETIC', 'AND', 'A', 'BIGOT', 'SHE', 'HAD', 'ONE', 'BEAUTY', 'THE', 'REST', 'OF', 'HER', 'PERSON', 'WAS', 'INDIFFERENTLY', 'FORMED', 'SHE', 'WAS', 'A', 'CLUMSY', 'COQUETTE', 'AND', 'A', 'CHASTE', 'ONE', 'HER', 'SKIN', 'WAS', 'WHITE', 'AND', 'FINE', 'SHE', 'DISPLAYED', 'A', 'GREAT', 'DEAL', 'OF', 'IT', 'IT', 'WAS', 'SHE', 'WHO'

In [72]:
feature = torch.Tensor(c.load_features()).unsqueeze(0)
batch = {
    "supervisions": {
        "text": [c.supervisions[0].text],
        "cut": [c],
        "sequence_idx": torch.arange(1),
        "start_frame": torch.zeros(1),
        "num_frames": torch.Tensor([feature.size(1)]).int(),
    },
    "inputs": feature,
}

In [73]:
feature = batch["inputs"]
assert feature.ndim == 3

supervisions = batch["supervisions"]
feature_lens = torch.Tensor([feature.size(1)]).int()
print(feature_lens, feature.shape)

tensor([1572], dtype=torch.int32) torch.Size([1, 1572, 80])


In [74]:
texts = batch["supervisions"]["text"]
y = sp.encode(texts, out_type=int)
y = k2.RaggedTensor(y)

cuts = batch['supervisions']['cut']
y_list = [libri_long_text_fst[tuple(get_uid_key(c.id)[:2])] for c in cuts]
# y_list = [libri_long_text[(6415, 100596)] for c in cuts]  # A mismatched text

In [75]:
targets = y_list
x = feature
x_lens = feature_lens

In [76]:
ctc_output, memory, memory_key_padding_mask = model(feature, supervisions)  # ctc_output is (N, T, C)
encoder_out_lens = torch.div(
    supervisions["num_frames"],
    params.subsampling_factor,
    rounding_mode="floor",
) - 1  # to fix some issues -- just discard some frames

In [82]:
def greedy(ctc_output):
    # Find the difference between consecutive elements
    diff = torch.diff(ctc_output)

    # Append a non-zero value to the start of the difference tensor
    diff = torch.cat((torch.tensor([1]), diff))

    # Find the indices of the non-zero elements
    indices = torch.nonzero(diff).squeeze()

    # Get the unique consecutive values
    unique = ctc_output[indices]

    # Remove all 0's
    result = unique[unique != 0]
    return result

In [83]:
print(c.id)
print(c.supervisions[0].text)
print(sp.encode(c.supervisions[0].text, out_type=int))
print(ctc_output.argmax(dim=-1))
print(sp.decode(greedy(ctc_output.argmax(dim=-1).squeeze()).tolist()))

3242-8112-0039-24096_sp0.9
SO TRIGGER SAID THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM SAID THE COMMISSIONER AROUND FORTY
[63, 56, 53, 353, 29, 115, 112, 356, 102, 287, 119, 97, 12, 3, 9, 156, 301, 41, 24, 12, 16, 414, 3, 7, 47, 356, 102, 126, 119, 97, 12, 409, 22, 26, 74, 4, 56, 229, 319, 133, 16, 9, 4, 129, 83, 188, 170, 241, 238, 47, 127, 6, 81, 101, 9, 199, 115, 4, 274, 17, 198, 3, 155, 29, 5, 87, 195, 42, 240]
tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,  63,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          56,  56, 181,   0, 353,   0,  29,   0,   0,   0, 115, 115,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0, 112,   0,   0,   0,   0, 356,   0,   0,   0,   0,
           0,   0, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [84]:
input_lengths = encoder_out_lens
batch_size = input_lengths.size(0)
# TODO
supervision_segments = torch.stack(
    (
        torch.arange(batch_size),
        torch.zeros(batch_size),
        input_lengths.cpu(),
    ),
    1,
).to(torch.int32)
indices = torch.argsort(supervision_segments[:, 2], descending=True)
supervision_segments = supervision_segments[indices]

In [ ]:
# 6531-61334-0104-1824_sp0.9
# ref: "JEHUA SHALMANESER DID NOT AGAIN ATTACK DAMASCUS HIS SPHERE OF INFLUENCE WAS THEREFORE CONFINED TO NORTH SYRIA"
#      "JEHU SHOWS THAT THE UNPRONOUNCED ALEPH AT THE END WAS AT THAT TIME SOUNDED JEHUA SHALMANESER DID NOT AGAIN ATTACK DAMASCUS HIS SPHERE OF INFLUENCE WAS THEREFORE CONFINED TO NORTH SYRIA"


In [88]:
# y_list = targets
# y_list = [get_decoding_graphs([texts[i]], sp)[0] for i in indices.tolist()]

tt = "THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM"
y_list = [get_decoding_graphs([tt], sp)[0] for i in indices.tolist()]

_y_list = [y_list[i] for i in indices.tolist()]
decoding_graph = k2.create_fsa_vec(_y_list)
decoding_graph = k2.arc_sort(decoding_graph)
decoding_graph = decoding_graph.to(device)
decoding_graph.shape[0], decoding_graph.num_arcs

(1, 252)

In [89]:
log_probs = ctc_output
lattice = get_lattice(
    nnet_output=log_probs,
    decoding_graph=decoding_graph,
    supervision_segments=supervision_segments,
    search_beam=params.search_beam,
    output_beam=params.output_beam,
    min_active_states=params.min_active_states,
    max_active_states=params.max_active_states,
    subsampling_factor=params.subsampling_factor,
)
lattice.shape[0], lattice.num_arcs

(1, 3908)

In [107]:
best_path = one_best_decoding(
    lattice=lattice,
    use_double_scores=True,
)
best_path.shape[0], best_path.num_arcs

(1, 0)

In [91]:
def get_str_by_range(c, rg):
    tlong = libri_long_text_str[tuple(get_uid_key(c.id)[:2])]
    return " ".join(tlong[rg[0]: rg[-1]])

In [93]:
hyps = get_texts(best_path)
print(f"hyps={hyps}")
print(f"[ref]: {c.supervisions[0].text}")
print(f"[hyp]: {sp.decode(hyps)[0]}")
# print(f"[hyp]: {get_str_by_range(c, hyps[0])}")

hyps=[[112, 356, 102, 287, 119, 97, 12, 3, 9, 156, 301, 41, 24, 12, 16, 414, 3, 7, 47, 356, 102, 126, 119, 97, 12, 409, 22, 26, 74, 4, 56, 229, 319, 133, 16, 9, 4, 129, 83, 188, 170, 241, 238, 47, 127, 6, 81, 101, 9, 199]]
[ref]: SO TRIGGER SAID THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM SAID THE COMMISSIONER AROUND FORTY
[hyp]: THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM


In [26]:
# https://github.com/k2-fsa/icefall/tree/master/egs/librispeech/ASR/conformer_ctc
# https://github.com/k2-fsa/icefall/blob/master/icefall/ali.py
# https://github.com/k2-fsa/icefall/pull/1482
# https://github.com/k2-fsa/icefall/pull/876

labels_ali = get_alignments(best_path, kind="labels")
aux_labels_ali = get_alignments(best_path, kind="aux_labels")

In [28]:
decoding_results = get_texts_with_timestamp(best_path)

In [31]:
decoding_results.timestamps

### Debug

In [112]:
'4340-15220-0052-15870'

'4340-15220-0052-15870'

In [98]:
rs = torch.load("/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg/scripts/temp.pt", map_location=torch.device('cpu'))

In [99]:
rs.keys()

dict_keys(['ctc_output', 'token_ids_indices', '_texts', 'token_ids'])

In [111]:
sp.decode(greedy(rs['ctc_output'][5].cpu().argmax(dim=-1).squeeze()).tolist())

'WHISPERED PNEIC BILLY WITH A CONSULING PAT ON THE PLAN OF THE SHSTHE WHEN THE CIRCLE IS COMES HERE YOU SHALL GO CERTAIN SHRE AND BETTERADY TO FEELING MEAN HE PROPOSED WHAT HE YOU THE HOLL MOCKER'

In [108]:
log_probs = rs['ctc_output'][5:6, ...]

# tt = rs['_texts'][5]
tt = 'WHISPERED KIND HEARTED BILLY WITH A CONSOLING PAT ON THE CROWN OF THE SHABBY STRAW HAT WHEN THE CIRCUS COMES HERE YOU SHALL GO CERTAIN SURE AND BETTY TOO SAID BEN FEELING MEAN WHILE HE PROPOSED WHAT HE KNEW WAS A HOLLOW MOCKERY'
y_list = [get_decoding_graphs([tt], sp)[0]]
decoding_graph = k2.create_fsa_vec(_y_list)
decoding_graph = k2.arc_sort(decoding_graph)
decoding_graph = decoding_graph.to(device)
decoding_graph.shape[0], decoding_graph.num_arcs

supervision_segments = torch.stack(
    (
        torch.tensor([0]),
        torch.tensor([0]),
        torch.tensor([1588]) // params.subsampling_factor,
    ),
    1,
).to(torch.int32)

In [124]:
lattice = get_lattice(
    nnet_output=log_probs,
    decoding_graph=decoding_graph,
    supervision_segments=supervision_segments,
    search_beam=params.search_beam + 80,
    output_beam=params.output_beam + 40,
    min_active_states=params.min_active_states * 5,
    max_active_states=params.max_active_states * 5,
    subsampling_factor=params.subsampling_factor,
)
lattice.shape[0], lattice.num_arcs

(1, 84701)

In [125]:
lattice = k2.connect(lattice)
lattice.shape[0], lattice.num_arcs

(1, 78288)

In [126]:
best_path = one_best_decoding(
    lattice=lattice,
    use_double_scores=True,
)
best_path.shape[0], best_path.num_arcs

(1, 398)

In [128]:
hyps = get_texts(best_path)
print(f"hyps={hyps}")
print(f"[ref]: {c.supervisions[0].text}")
print(f"[hyp]: {sp.decode(hyps)[0]}")
# print(f"[hyp]: {get_str_by_range(c, hyps[0])}")

hyps=[[112, 356, 102, 287, 119, 97, 12, 3, 9, 156, 301, 41, 24, 12, 16, 414, 3, 7, 47, 356, 102, 126, 119, 97, 12, 409, 22, 26, 74, 4, 56, 229, 319, 133, 16, 9, 4, 129, 83, 188, 170, 241, 238, 47, 127, 6, 81, 101, 9, 199]]
[ref]: SO TRIGGER SAID THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM SAID THE COMMISSIONER AROUND FORTY
[hyp]: THEY MIGHT HAVE TWO THIRDS OF WHAT EVERYBODY WANTS AND YOU MIGHT HAVE ONE THIRD RIGHT HERE ON THE TABLE HOW MANY OF THE LATER RAIDERS DID YOU CATCH ALL OF THEM


In [101]:
rs['_texts'][5]

'WHISPERED KIND HEARTED BILLY WITH A CONSOLING PAT ON THE CROWN OF THE SHABBY STRAW HAT WHEN THE CIRCUS COMES HERE YOU SHALL GO CERTAIN SURE AND BETTY TOO SAID BEN FEELING MEAN WHILE HE PROPOSED WHAT HE KNEW WAS A'